In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import RandomizedSearchCV
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)
properati=properati.dropna().reset_index()

<IPython.core.display.Javascript object>

In [9]:
clf = svm.SVC()
scaler = StandardScaler()
scaler2= MinMaxScaler()
cant_buckets = 10

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)

# tenia un nan nose porque
properati.dropna(inplace=True)
# lo casteo a float porque si no tira error
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 
properati.groupby('categories_by_price').count()

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0.0,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
1.0,367,367,367,367,367,367,367,367,367,367,...,367,367,367,367,367,367,367,367,367,367
2.0,911,911,911,911,911,911,911,911,911,911,...,911,911,911,911,911,911,911,911,911,911
3.0,633,633,633,633,633,633,633,633,633,633,...,633,633,633,633,633,633,633,633,633,633
4.0,309,309,309,309,309,309,309,309,309,309,...,309,309,309,309,309,309,309,309,309,309
5.0,96,96,96,96,96,96,96,96,96,96,...,96,96,96,96,96,96,96,96,96,96
6.0,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
7.0,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
8.0,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11


# Random Search

In [10]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

X=scaler.fit_transform(X,y)
X=scaler2.fit_transform(X,y)
#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


In [13]:
C_range = np.arange(0.1,1000,0.1)
gamma_range = np.arange(10,20,1)

In [15]:
%%notify

from sklearn.model_selection import StratifiedShuffleSplit

param_grid ={"gamma": gamma_range, "C": C_range}
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3) #Se cuelga si lo pones
random_search = RandomizedSearchCV(clf,param_distributions=param_grid,n_iter=20,cv=5,n_jobs=-1)
start = time()
random_search.fit(X, y)

print("RandomizedSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(random_search.cv_results_['params'])))
score.report_single(random_search.cv_results_)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

KeyboardInterrupt: 